# LLaMA-3 Tokenizer  
# 1. Import

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "allganize/Llama-3-Alpha-Ko-8B-Evo"

In [ ]:
model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# 2. Encode & Decode

In [ ]:
llm_input = [
    # {"Card_Name":"삼성카드 & MILEAGE PLATINUM(스카이패스)","benefits":"3% 캐시백, 영화 할인"},
    {"Card_Name":"KB국민 와이즈카드", "benefits":"뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아"}
]

# 3. Template  
- user : 사용자 신분 : 사용자가 챗봇에게 질문할 내용이 됩니다.  
- system : LLM에게 시키고 싶은 것 : LLM이 어떤 기준을 가지고 작동할지에 대한 설정  
- assistant : LLM의 답변  


In [ ]:
messages = [
    {"role": "system", "content":"당신은 광고 카피라이터입니다. 입력된 정보에 기반하여 기발하고 명확한 카피라이팅 문구를 제작하세요."},
    {"role" :"user", "content" : llm_input}
]

# 4. ChatTemplate 인코딩 & 디코딩

In [ ]:
# ChatTemplate 만들기
template_messages = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors= 'pt' # 파이토치 텐서로 리턴
    )
print(template_messages)

tensor([[128000, 128006,   9125, 128007,    271,  65895,  83628,  34804, 112601,
         103236, 102477, 108157,  34961,  80052,     13,  43449,  53400,  61139,
          19954, 126470,  83290,  55216, 102133, 101360, 104167, 111372,  24486,
         103236, 102477, 108157, 105876,  54535,  89359,  18918, 114699,  92245,
             13, 128009, 128006,    882, 128007,    271,     58,  13922,   5889,
          19552,   1232,    364, 111690,  33931, 101436,  30446,    612,    386,
           3015,   3669,  10528,    835,    691,   2864,      7,  25941, 101436,
          13094, 108507,  25941,  19255,    364,  68244,   1220,   1232,    364,
             18,      4, 111031,  30426, 106113,     11, 110243, 125069,  25762,
           5473,   5889,  19552,   1232,    364,  30962, 100654, 101607,  75984,
         103618, 102668, 101436,  30446,    518,    364,  68244,   1220,   1232,
            364, 114448, 102199,     11,  38295,    102,  64189,     11, 103236,
         104249,     14, 105

# 5. LLaMA-3 모델 호출하기

In [ ]:
# 비트 양자화를 위한 패키지입니다. 모델의 용량을 줄여주는 역할을 한다.
# 설치 후 세션 다시 시작
# !pip install -q -U bitsandbytes

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

allganize/Llama-3-Alpha-Ko-8B-Instruct


config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [ ]:
device = 'cuda'

# 모델에 입력할 데이터를 gpu로 옮긴다.
model_inputs = template_messages.to(device)
model_inputs

tensor([[128000, 128006,   9125, 128007,    271,  65895,  83628,  34804, 112601,
         103236, 102477, 108157,  34961,  80052,     13,  43449,  53400,  61139,
          19954, 126470,  83290,  55216, 102133, 101360, 104167, 111372,  24486,
         103236, 102477, 108157, 105876,  54535,  89359,  18918, 114699,  92245,
             13, 128009, 128006,    882, 128007,    271,     58,  13922,   5889,
          19552,   1232,    364, 111690,  33931, 101436,  30446,    612,    386,
           3015,   3669,  10528,    835,    691,   2864,      7,  25941, 101436,
          13094, 108507,  25941,  19255,    364,  68244,   1220,   1232,    364,
             18,      4, 111031,  30426, 106113,     11, 110243, 125069,  25762,
           5473,   5889,  19552,   1232,    364,  30962, 100654, 101607,  75984,
         103618, 102668, 101436,  30446,    518,    364,  68244,   1220,   1232,
            364, 114448, 102199,     11,  38295,    102,  64189,     11, 103236,
         104249,     14, 105

In [ ]:
# 답변 종료 토큰 설정
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

### **답변 생성하기** : LLM에서 답변을 받기 위해서는 generate 함수 사용

In [ ]:
# 모델 입력
generated_ids = model.generate(
    model_inputs, # 정수 인코딩이 된 프롬프트(chat Template)
    max_new_tokens=300, # 모델이 대답할 응답 메시지의 최대 길이
    eos_token_id = terminators, # 모델의 텍스트 생성 종료 토큰
    repetition_penalty = 1.05 # 반복된 응답 지수. 값이 작아질 수록 반복된 응답을 수행
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
# batch_decode : 여러 개의 input에 대해 한꺼번에 변환
decoded = tokenizer.batch_decode(generated_ids)
decoded[0]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n당신은 광고 카피라이터입니다. 입력된 정보에 기반하여 기발하고 명확한 카피라이팅 문구를 제작하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n[{'Card_Name': '삼성카드 & MILEAGE PLATINUM(스카이패스)', 'benefits': '3% 캐시백, 영화 할인'}, {'Card_Name': 'KB국민 와이즈카드', 'benefits': '뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아'}]<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n여러 카드의 다양한 혜택을 소개하는 광고 카피입니다. 아래는 그 예시입니다.\n\n**삼성카드 & MILEAGE PLATINUM(스카이패스)**\n지금까지 스카이패스를 즐기고 싶었나요? 삼성카드 & MILEAGE PLATINUM(스카이패스)로 3% 캐시백과 영화 할인 혜택을 누리세요! 스카이패스 회원이 되면, 매월 3%의 캐시백을 받을 수 있으며, 영화관에서 할인 혜택도 누릴 수 있습니다. 더 이상 지갑을 두드리지 않으셔도 됩니다!\n\n**KB"

# 6. RAG 시스템 결합

In [ ]:
!pip -q install langchain pypdf chromadb sentence-transformers faiss-gpu langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 88.2 MB/s eta 0:00:

# 파이프라인 정의  
- 랭체인에서 사용할 내용 : 허깅페이스 오픈 LLM을 연결할 파이프라인을 정의  
- 허깅페이스의 파이프라인을 생성해서 랭체인의 파이프라인으로 이어주기

In [ ]:
이모티콘, 장원영 gpt
gpt 사용하기

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline # 오픈 모델 체이닝
from langchain.chains import LLMChain

# 1. 허깅페이스 파이프라인 정의. LLM은 생성 모델이기 때문에 text-generation 파이프라인을 생성
text_generation_pipeline = pipeline(
    model = model, # 사용할 모델. 여기서는 LLaMA-3 모델
    tokenizer = tokenizer, # LLaMA-3의 토크나이저
    task = 'text-generation',
    return_full_text=False, # 답변만 생성 결과로 받고 싶은 경우 사용. True로 설정하면 입력 프롬프트까지 모두 나옴
    max_new_tokens=300,
    temperature=0.7, # 응답의 창의성 조정
    top_p=0.9 # 생성 확률의 누적값 기반으로 다양성 제어
)


#  프롬프트 템플릿 직접 정의
prompt_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 광고 카피라이터입니다. 주어진 카드 정보와 혜택을 바탕으로 기발하고 명확한 카피라이팅 문구를 제작하는 것이 역할입니다.
당신의 문구는 사용자에게 카드의 가치를 한눈에 전달할 수 있어야 하며, 읽는 사람이 혜택을 즉각적으로 이해하고 매력을 느끼게 해야 합니다.

<|eot_id|><|start_header_id|>user<|end_header_id|>

카드 정보:
{llm_input}

요청사항:
입력된 카드 정보 각각에 대해 2~3개의 기발한 광고 문구를 작성해주세요.

예시 문구:
1. "영화와 캐시백의 만남 – 삼성카드 MILEAGE PLATINUM이 선사합니다!"
2. "와이즈한 선택, 국민 와이즈카드로 누리는 뷰티와 문화의 향연!"

위와 같은 스타일로 작성해주세요.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# 2. Langchain 파이프라인 정의
# 허깅페이스 파이프라인 사용
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 3. Langchain의 프롬프트 템플릿 생성
prompt = PromptTemplate(
    input_variables=["llm_input"],
    template= prompt_template
)

# llm chain 구축
llm_chain = LLMChain(llm=llm, prompt=prompt)

llm_input = """
- 카드 이름: KB국민 와이즈카드
- 혜택: 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아
"""

# 실행
response = llm_chain.run({"llm_input":llm_input})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
print(response)

여러분의 선택에 따라, 국민 와이즈카드의 혜택을 소개합니다.

1. "뷰티와 납부의 조화, 국민 와이즈카드로 더 아름다워진 당신!"
2. "카페에서 커피 한 잔, 국민 와이즈카드로 더 즐거운 시간을 누리세요!"
3. "쇼핑과 간편결제의 만남, 국민 와이즈카드로 더 쉽고 편리하게 즐기세요!"
4. "의료와 교통의 걱정, 국민 와이즈카드로 더 안전하게 해결하세요!"
5. "문화와 교육의 향연, 국민 와이즈카드로 더 풍성한 삶을 누리세요!"
6. "육아와 교육의 걱정, 국민 와이즈카드로 더 편리하게 해결하세요!"
7. "주유와 교통의 걱정, 국민 와이즈카드로 더 안전하게 해결하세요!"
8. "문화와 교통의 만남, 국민 와이즈카드로 더 풍성한 삶을 누리세요!"
9. "뷰티와 문화의 만남, 국민 와이즈카드로 더 아름다워진 당신!"
10. "카페에서 커피 한 잔, 국민 와이즈카드로 더 즐거운 시간을 누리세요!"
11. "쇼핑과 간편결제


# 6. RAG 시스템 결합  
## 6.1 RAG를 위한 VectorDB 구축

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.schema.runnable import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
# CSV 로더 생성
loader = CSVLoader(file_path="/content/drive/MyDrive/CARD-LANGCHAIN/data/final_ad_copywriting_category_data_kr.csv")

# 데이터 로드
docs = loader.load()

print(len(docs))
print(docs[0].metadata)

27
{'source': '/content/drive/MyDrive/CARD-LANGCHAIN/data/final_ad_copywriting_category_data_kr.csv', 'row': 0}


In [ ]:
print(docs[0].page_content)
print(len(docs))

카테고리: 항공
상세 카테고리: 항공마일리지
광고 문구: 항공마일리지로 누리는 특별한 혜택 – 항공 카드를 지금 확인하세요!
27


In [ ]:
for doc in docs[:5]:
  print("문서 내용:", doc.page_content)
  print("메타데이터:",doc.metadata)

문서 내용: 카테고리: 항공
상세 카테고리: 항공마일리지
광고 문구: 항공마일리지로 누리는 특별한 혜택 – 항공 카드를 지금 확인하세요!
메타데이터: {'source': '/content/drive/MyDrive/CARD-LANGCHAIN/data/final_ad_copywriting_category_data_kr.csv', 'row': 0}
문서 내용: 카테고리: 쇼핑/간편결제
상세 카테고리: 쇼핑
광고 문구: 쇼핑의 가치를 높이다 – 더 많은 포인트로 당신의 일상을 풍요롭게!
메타데이터: {'source': '/content/drive/MyDrive/CARD-LANGCHAIN/data/final_ad_copywriting_category_data_kr.csv', 'row': 1}
문서 내용: 카테고리: 쇼핑/간편결제
상세 카테고리: 쇼핑
광고 문구: 작은 지출, 큰 기쁨 – 쇼핑 후 돌아오는 포인트의 즐거움!
메타데이터: {'source': '/content/drive/MyDrive/CARD-LANGCHAIN/data/final_ad_copywriting_category_data_kr.csv', 'row': 2}
문서 내용: 카테고리: 쇼핑/간편결제
상세 카테고리: 간편결제
광고 문구: 결제의 새 시대 – 간편결제로 빠르고 똑똑하게 혜택을 누리세요.
메타데이터: {'source': '/content/drive/MyDrive/CARD-LANGCHAIN/data/final_ad_copywriting_category_data_kr.csv', 'row': 3}
문서 내용: 카테고리: 쇼핑/간편결제
상세 카테고리: 간편결제
광고 문구: 간편함 그 이상의 혜택 – 쇼핑할 때마다 쌓이는 보너스를 경험하세요.
메타데이터: {'source': '/content/drive/MyDrive/CARD-LANGCHAIN/data/final_ad_copywriting_category_data_kr.csv', 'row': 4}


## 6.2 RAG Chain 생성

In [ ]:
# 허깅페이스 임베딩 모델을 Langchain으로 연결
model_name = "BAAI/bge-m3"

# 허깅페이스 임베딩 모델을 Langchain화 시켜주기
hf = HuggingFaceEmbeddings(model_name=model_name)

<ipython-input-38-c8eb37facf54>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceEmbeddings(model_name=model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
vector_db = FAISS.from_documents(docs, hf)
retriever = vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

In [ ]:
# vector db 테스트
search_result = retriever.get_relevant_documents("혜택: 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아")
search_result

<ipython-input-41-0c459917b175>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  search_result = retriever.get_relevant_documents("혜택: 뷰티, 납부, 카페/베이커리, 쇼핑/간편결제, 의료, 교통/주유, 문화, 교육/육아")


[Document(metadata={'source': '/content/drive/MyDrive/CARD-LANGCHAIN/data/final_ad_copywriting_category_data_kr.csv', 'row': 15}, page_content='카테고리: 뷰티\n상세 카테고리: 뷰티\n광고 문구: 뷰티로 누리는 특별한 혜택 – 뷰티 카드를 지금 확인하세요!'),
 Document(metadata={'source': '/content/drive/MyDrive/CARD-LANGCHAIN/data/final_ad_copywriting_category_data_kr.csv', 'row': 26}, page_content='카테고리: 납부\n상세 카테고리: 납부 혜택\n광고 문구: 납부 혜택로 누리는 특별한 혜택 – 납부 카드를 지금 확인하세요!'),
 Document(metadata={'source': '/content/drive/MyDrive/CARD-LANGCHAIN/data/final_ad_copywriting_category_data_kr.csv', 'row': 6}, page_content='카테고리: 쇼핑/간편결제\n상세 카테고리: 편의점\n광고 문구: 편의점로 누리는 특별한 혜택 – 쇼핑/간편결제 카드를 지금 확인하세요!')]

In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/CARD-LANGCHAIN/data/final_ad_copywriting_category_data_kr.csv")

In [ ]:
data

,카테고리,상세 카테고리,광고 문구
0,항공,항공마일리지,항공마일리지로 누리는 특별한 혜택 – 항공 카드를 지금 확인하세요!
1,쇼핑/간편결제,쇼핑,쇼핑의 가치를 높이다 – 더 많은 포인트로 당신의 일상을 풍요롭게!
2,쇼핑/간편결제,쇼핑,"작은 지출, 큰 기쁨 – 쇼핑 후 돌아오는 포인트의 즐거움!"
3,쇼핑/간편결제,간편결제,결제의 새 시대 – 간편결제로 빠르고 똑똑하게 혜택을 누리세요.
4,쇼핑/간편결제,간편결제,간편함 그 이상의 혜택 – 쇼핑할 때마다 쌓이는 보너스를 경험하세요.
5,쇼핑/간편결제,포인트/캐시백,포인트/캐시백로 누리는 특별한 혜택 – 쇼핑/간편결제 카드를 지금 확인하세요!
6,쇼핑/간편결제,편의점,편의점로 누리는 특별한 혜택 – 쇼핑/간편결제 카드를 지금 확인하세요!
7,쇼핑/간편결제,대형마트,대형마트로 누리는 특별한 혜택 – 쇼핑/간편결제 카드를 지금 확인하세요!
8,카페/베이커리,카페,커피 한 잔의 여유를 더 달콤하게 – 카페 할인 혜택을 놓치지 마세요.
9,카페/베이커리,카페,따뜻한 커피처럼 – 카페에서의 순간도 혜택으로 가득 채우세요.


## 6.3 Rag와 LLM chain 연결

In [ ]:
def rag_process(query):
  results = retriever.get_relevant_documents(query)

  llm_input = "\n".join([doc.page_content for doc in results])

  return llm_chain.run({"llm_input":llm_input})

query = "KB 국민 와이즈카드와 관련된 광고 문구를 작성해주세요"
response = rag_process(query)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


카드 정보에 맞는 광고 문구를 작성해드리겠습니다.

1. 문화 카드
- "영화로 누리는 특별한 혜택, 문화 카드가 선사합니다!"
- "문화의 향연을 즐기세요, 문화 카드가 준비해 있습니다!"
- "문화 카드, 영화와 함께 특별한 경험을 선사합니다!"

2. 뷰티 카드
- "뷰티로 누리는 특별한 혜택, 뷰티 카드가 선사합니다!"
- "뷰티와 문화의 만남, 뷰티 카드가 준비해 있습니다!"
- "뷰티 카드, 뷰티와 함께 특별한 경험을 선사합니다!"

3. 문화 카드
- "문화 카드, 문화의 향연을 즐기세요!"
- "문화 카드, 특별한 경험을 선사합니다!"
- "문화 카드, 문화와 함께 특별한 경험을 선사합니다!"

이와 같은 문구는 카드의 혜택을 명확하게 전달하고, 사용자에게 특별한 경험을 선사하는 카드의 매력을 전달하고자 합니다. 또한, 카드의 카테고리와 상세 카테고리를 고려하여 카드의 특성을 강조하고 있습니다. 
